In [23]:
from pyramid.arima import auto_arima
import pandas as pd
import numpy as np

In [24]:
data = pd.read_csv('/home/affine/Koch/auto_arima/Finance_Final_data3.csv')

In [25]:
data.shape

(132, 2258)

In [26]:
data.columns

Index(['year', 'month', 'High1222.HK', 'Low1222.HK', 'Open1222.HK',
       'Close1222.HK', 'High4715.KL', 'Low4715.KL', 'Open4715.KL',
       'Close4715.KL',
       ...
       'OpenX.TO', 'CloseX.TO', 'HighYTRGF', 'LowYTRGF', 'OpenYTRGF',
       'CloseYTRGF', 'HighZARFF', 'LowZARFF', 'OpenZARFF', 'CloseZARFF'],
      dtype='object', length=2258)

In [27]:
DV = 'HighGLGI'

In [28]:
data['HighGLGI'].isna().sum()

0

In [29]:
data.shape

(132, 2258)

In [30]:
train = data[DV][0:100]
test = data[DV][100:]

In [31]:
print(train.isna().sum())
print(test.isna().sum())

0
0


In [32]:
stepwise_model = auto_arima(train, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=-257.515, BIC=-245.185, Fit time=1.529 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=-210.843, BIC=-205.911, Fit time=0.065 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=-245.890, BIC=-236.027, Fit time=0.945 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=-258.295, BIC=-248.431, Fit time=0.943 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=-256.814, BIC=-244.484, Fit time=1.548 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=-217.742, BIC=-210.344, Fit time=0.135 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=-252.629, BIC=-245.231, Fit time=0.674 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 1, 1, 

In [33]:
print(stepwise_model.aic())

-260.9594774903298


In [34]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 3),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [35]:
test.shape

(32,)

In [36]:
future_forecast = stepwise_model.predict(n_periods=32)

In [37]:
type(test)

pandas.core.series.Series

In [38]:
test_data = pd.DataFrame({'actual': test, 'predicted': future_forecast})

In [39]:
test_data

actual  predicted
100  0.292000   0.251139
101  0.295455   0.262387
102  0.297273   0.277480
103  0.291905   0.283280
104  0.248095   0.273343
105  0.227727   0.243081
106  0.220500   0.225298
107  0.191818   0.205674
108  0.194737   0.231491
109  0.200000   0.251294
110  0.290000   0.274983
111  0.241905   0.279604
112  0.255238   0.243211
113  0.245909   0.258280
114  0.265500   0.266415
115  0.273043   0.271804
116  0.243810   0.261457
117  0.245714   0.230785
118  0.238095   0.212592
119  0.232381   0.192558
120  0.235500   0.217965
121  0.232105   0.237358
122  0.232609   0.260637
123  0.275789   0.264847
124  0.312273   0.228044
125  0.336364   0.242703
126  0.357000   0.250428
127  0.389130   0.255407
128  0.440000   0.244650
129  0.448636   0.213568
130  0.460952   0.194965
131  0.477000   0.174521

In [40]:
test_data['APE'] = abs(test_data['actual'] - test_data['predicted'])/test_data['actual']

In [41]:
print('Mape is :', np.mean(test_data['APE']))

Mape is : 0.1689920988773037
